In [19]:
import os
import numpy as np
from tqdm.auto import tqdm

# 두 디렉터리 경로 설정
cpu_result_folder = "C:/Users/01sun/source/repos/raspberrypi/trace2/alignment_and_drop_cpu"
gpu_result_folder = "C:/Users/01sun/source/repos/raspberrypi/trace2/alignment_and_drop_GPU"
# 비교 허용 오차 설정
cpu_files = sorted([f for f in os.listdir(cpu_result_folder) if f.startswith("alignTrace")])
gpu_files = sorted([f for f in os.listdir(gpu_result_folder) if f.startswith("alignTrace")])
# 첫 10개 파일만 출력
print("CPU 파일 리스트 (처음 10개):", cpu_files[0])
print("GPU 파일 리스트 (처음 10개):", gpu_files[0])
# 폴더 내 파일 개수 출력
print(f"CPU 결과 파일 개수: {len(cpu_files)}")
print(f"GPU 결과 파일 개수: {len(gpu_files)}")

# 파일 개수 불일치 여부 확인
if len(cpu_files) != len(gpu_files):
    print("경고: CPU와 GPU 결과 폴더의 파일 개수가 다릅니다!")

# 결과 비교
same_count = 0
different_count = 0
different_files = []

for cpu_file, gpu_file in tqdm(zip(cpu_files, gpu_files), total=len(cpu_files)):
    cpu_path = os.path.join(cpu_result_folder, cpu_file)
    gpu_path = os.path.join(gpu_result_folder, gpu_file)

    cpu_trace = np.load(cpu_path)
    gpu_trace = np.load(gpu_path)
    
    if not np.array_equal(cpu_trace, gpu_trace):
        print(f"Different file name: {cpu_file}, {gpu_file}")
        different_files.append(gpu_file)
        different_count +=1
    else:
        same_count += 1
# 비교 결과 출력
print(f"결과가 같은 파일 개수: {same_count}")
print(f"결과가 다른 파일 개수: {different_count}")

CPU 파일 리스트 (처음 10개): alignTrace100000.npy
GPU 파일 리스트 (처음 10개): alignTrace100000.npy
CPU 결과 파일 개수: 151318
GPU 결과 파일 개수: 151318


  0%|          | 0/151318 [00:00<?, ?it/s]

Different file name: alignTrace433530.npy, alignTrace433530.npy
Different file name: alignTrace609147.npy, alignTrace609147.npy
Different file name: alignTrace869600.npy, alignTrace869600.npy
결과가 같은 파일 개수: 151315
결과가 다른 파일 개수: 3
